In [15]:
import numpy as np
import math
import random

## Testovací funkce

In [1]:
# 1st DeJong function

def dejong1(x):
    return sum([i**2 for i in x])

# 2st DeJong function
# f2(x)=sum(100·(x(i+1)-x(i)^2)^2+(1-x(i))^2)
#   i=1:n-1; -2.048<=x(i)<=2.048.
def dejong2(x):
    d = len(x)
    suma = 0
    i=0
    for i in range(d-1):
        xi = x[i]
        xnext = x[i+1]
        new = 100 * (xnext - xi**2)**2 + (xi-1)**2
        suma += new
    return suma 

# Schweffel function
def schweffel(x):
    d = len(x)
    suma = 0
    for i in range(d):
        xi = x[i]
        suma += xi * math.sin(math.sqrt(abs(xi)))
    return (418.9829 * d - suma)
    
    

## Algoritmy

In [ ]:
# Random Search

def random_search(n_iterations, cost_func, dimensions ):
    best_solution = None
    best_fitness = math.inf

    for i in range(n_iterations):
        solution = [np.random.uniform(-5.12, 5.12) for i in range(dimensions)]
        fitness = cost_func(solution)

        if fitness < best_fitness:
            best_solution = solution
            best_fitness = fitness
            
    return (best_solution, best_fitness)


# Simulované žíhání

def simulated_annealing(n_iterations, cost_function, dimensions, n_neighbours, area, max_temp, min_temp, cooling_decr):
    starting_vector = np.random.uniform(-area, area, dimensions)
    current_solution = starting_vector
    current_fitness = cost_function(current_solution)
    history = [current_fitness]
    T = max_temp

    for i in range(n_iterations):
        neighbours = generate_neighbors(current_solution, n_neighbours, area)
        for neighbour_solution in neighbours:
            neighbour_fitness = cost_function(neighbour_solution) 
            delta_f = neighbour_fitness - current_fitness
            if(delta_f < 0):
                current_fitness = neighbour_fitness
                current_solution = neighbour_solution
            elif(neighbour_fitness < current_fitness):
                current_fitness = neighbour_fitness
                current_solution = neighbour_solution
            elif(random.random() < math.exp(-delta_f/T)): #volani metropolise
                current_fitness = neighbour_fitness
                current_solution = neighbour_solution
        history.append(current_fitness)
        T = max_temp*cooling_decr^i
        if(T < min_temp):
            T = min_temp
    return history


def generate_neighbors(current_solution, n_neighbors, area):
    neighbors = []
    min_value = -area
    max_value = area
    step_10_percent = 0.1 * (max_value - min_value)
    sigma = step_10_percent/2
    for i in range(n_neighbors):
        neighbor = current_solution + np.random.normal(0, sigma, len(current_solution))
        while not all((neighbor >= -(area)) & (neighbor <= area)):
            neighbor = current_solution + np.random.normal(0, sigma, len(current_solution))
        neighbors.append(neighbor)
    return neighbors


## Výseldky

In [ ]:
def generate_output(range, dimension, obj_func, sigma, population, num_iterations, title):
    T0 = 1000
    Tf = 1e-5

In [29]:
random.random()

0.42695561433801743